## Import packages and modules

In [21]:
from models.tda_models import TDA_PI34_Model, TDA_PI42_Model, VECTOR_STITCHING_PI_Model_34, VECTOR_STITCHING_PI_Model_42
from models.raw_models import Raw_Model, Dummy_Model
from data_preprocessing import process_mnist_alike

import tensorflow as tf


In [22]:
# models

raw_model = Raw_Model() # cnn models working on raw images
tda_model_34 = TDA_PI34_Model() # cnn model working on persistance images
vector_stitching_model_34 = VECTOR_STITCHING_PI_Model_34() # cnn model working on stitched raw and PI images


In [11]:
# indices for 10 fold cross validation

indices = [
    ((32427, 32927), (36258, 36358))
]

indices100 = [
    ((8028, 8128), (2237, 2337)),
    ((31081, 31181), (47635, 47735)),
    ((51438, 51538), (24766, 24866)),
    ((32427, 32527), (36258, 36358)),
    ((0, 100), (17901, 18001)),
    ((49216, 49316), (13836, 13936)),
    ((51207, 51307), (100, 200)),
    ((48137, 48237), (57296, 57396)),
    ((1000, 1100), (59758, 59858)),
    ((29740, 29840), (17885, 17985))
]

indices250 = [
    ((8028, 8278), (2237, 2337)),
    ((31081, 31331), (47635, 47735)),
    ((51438, 51688), (24766, 24866)),
    ((32427, 32677), (36258, 36358)),
    ((0, 250), (17901, 18001)),
    ((49216, 49466), (13836, 13936)),
    ((51207, 51457), (100, 200)),
    ((48137, 48387), (57296, 57396)),
    ((1000, 1250), (59758, 59858)),
    ((29740, 29990), (17885, 17985))
]

indices500 = [
    ((8028, 8528), (2237, 2337)),
    ((31081, 31581), (47635, 47735)),
    ((51438, 51938), (24766, 24866)),
    ((32427, 32927), (36258, 36358)),
    ((0, 500), (17901, 18001)),
    ((49216, 49716), (13836, 13936)),
    ((51207, 51707), (100, 200)),
    ((48137, 48637), (57296, 57396)),
    ((1000, 1500), (59758, 59858)),
    ((29740, 30240), (17885, 17985))
]


indices1000 = [
    ((8028, 9028), (2237, 2337)),
    ((31081, 32081), (47635, 47735)),
    ((51438, 52438), (24766, 24866)),
    ((32427, 33427), (36258, 36358)),
    ((0, 1000), (17901, 18001)),
    ((49216, 50216), (13836, 13936)),
    ((51207, 52207), (100, 200)),
    ((48137, 49137), (57296, 57396)),
    ((1000, 2000), (59758, 59858)),
    ((29740, 30740), (17885, 17985))
]





In [12]:
(X, y), _ = tf.keras.datasets.mnist.load_data()

tda_history = []
raw_history = []
vector_stitching_history = []
j=0
_verbose = 0
early_stopping_tda = 5
early_stopping_raw = 5
early_stopping_vector_stitching = 5

for i in indices100:
    data = process_mnist_alike(data=(X,y), training_indices=i[0], testing_indices=i[1], dist_ratio=10)
    
    #TDA model
    #early stopping
    tf.keras.backend.clear_session()
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=early_stopping_tda, restore_best_weights=True)
    tda_model_34.model.fit(data["X_train_clean_tda_good_34"], data["y_train"], epochs=20, batch_size=1, validation_data=(data["X_test_noisy_tda_good_34"], data["y_test"]), callbacks=[early_stopping], verbose=_verbose)
    #print the best validation accuracy
    print("krok: ", j)
    print("Best validation accuracy for TDA model 34: \t", max(tda_model_34.model.history.history['val_accuracy']))
    tda_history.append(max(tda_model_34.model.history.history['val_accuracy']))
    

    # RAW model
    tf.keras.backend.clear_session()
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=early_stopping_raw, restore_best_weights=True)
    raw_model.model.fit(data["X_train"], data["y_train"], epochs=20, batch_size=1, validation_data=(data["X_test_noisy_random"], data["y_test"]), callbacks=[early_stopping], verbose=_verbose)
    print("Best validation accuracy for RAW model: \t", max(raw_model.model.history.history['val_accuracy']))
    raw_history.append(max(raw_model.model.history.history['val_accuracy']))


    
    #Vector stitching model
    tf.keras.backend.clear_session()
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=early_stopping_vector_stitching, restore_best_weights=True)
    vector_stitching_model_34.model.fit(data["X_train_clean_vector_stitching_good_34"], data["y_train"], epochs=20, batch_size=1, validation_data=(data["X_test_noisy_vector_stitching_good_34"], data["y_test"]), callbacks=[early_stopping], verbose=_verbose)
    print("Best validation accuracy for VS model 34: \t", max(vector_stitching_model_34.model.history.history['val_accuracy']))
    vector_stitching_history.append(max(vector_stitching_model_34.model.history.history['val_accuracy']))
    

    #reload models
    raw_model = Raw_Model() # cnn models working on raw images
    tda_model_34 = TDA_PI34_Model() # cnn model working on persistance images
    vector_stitching_model_34 = VECTOR_STITCHING_PI_Model_34() # cnn model working on stitched raw and PI images

    
    j=j+1

c:\Users\jawor\Desktop\Projekt_licencjacki\data_preprocessing.py:73: FutureWarning: `seed` is a deprecated argument name for `random_noise`. It will be removed in version 0.23. Please use `rng` instead.
  X_train_noisy = random_noise(X_train, mode="s&p",amount=0.05, seed=666)
c:\Users\jawor\Desktop\Projekt_licencjacki\data_preprocessing.py:74: FutureWarning: `seed` is a deprecated argument name for `random_noise`. It will be removed in version 0.23. Please use `rng` instead.
  X_test_noisy = random_noise(X_test, mode="s&p",amount=0.05, seed=666)


1/4
2/4
inside CombineTDAWithRawImages_34
(500, 34, 28, 28)
after normalization
(500, 34, 28, 28)
0.0
1.0
inside CombineTDAWithRawImages_34
(100, 34, 28, 28)
after normalization
(100, 34, 28, 28)
0.0
1.0
3/4
inside CombineTDAWithRawImages_34
(500, 34, 28, 28)
after normalization
(500, 34, 28, 28)
0.0
1.0
inside CombineTDAWithRawImages_34
(100, 34, 28, 28)
after normalization
(100, 34, 28, 28)
0.0
1.0
4/4
krok:  0
Best validation accuracy for TDA model 34: 	 0.3700000047683716
Best validation accuracy for RAW model: 	 0.11999999731779099
Best validation accuracy for VS model 34: 	 0.7699999809265137


In [22]:

# print means of history
print("TDA mean: ", sum(tda_history)/len(tda_history))
print("RAW mean: ", sum(raw_history)/len(raw_history))
print("Vector stitching mean: ", sum(vector_stitching_history)/len(vector_stitching_history))


TDA mean:  0.31099999994039534
RAW mean:  0.16599999889731407
Vector stitching mean:  0.4050000011920929


In [23]:
print("tda model:")
for i in tda_history:
    print(i)
print("\nraw model:")
for i in raw_history:
    print(i)
print("\nvector stitching model:")
for i in vector_stitching_history:
    print(i)


tda model:
0.3499999940395355
0.3700000047683716
0.20000000298023224
0.36000001430511475
0.25
0.38999998569488525
0.23000000417232513
0.2800000011920929
0.44999998807907104
0.23000000417232513

raw model:
0.09000000357627869
0.14000000059604645
0.20999999344348907
0.07999999821186066
0.20000000298023224
0.25999999046325684
0.1599999964237213
0.18000000715255737
0.23999999463558197
0.10000000149011612

vector stitching model:
0.4699999988079071
0.5099999904632568
0.27000001072883606
0.41999998688697815
0.38999998569488525
0.2800000011920929
0.3400000035762787
0.49000000953674316
0.5400000214576721
0.3400000035762787
